In [1]:
#设置列对其
import pandas as pd
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)

In [2]:
#信息熵计算函数
import numpy as np
from math import log

def entropy(ele):
    '''
    input: 包含类别取值的列表
    output: 信息熵
    '''
    probs = [ele.count(i)/len(ele) for i in set(ele)]
    
    entropy = -sum([prob*log(prob,2) for prob in probs])
    
    return entropy

In [3]:
#根据数据集和指定特征定义数据集划分函数
def df_split(df, featrue):
    '''
    输入:
    df: 待划分的训练数据
    col: 划分数据的依据特征——标签
    输出:
    res_dict: 根据特征取值划分后的不同数据集的字典
    '''
    unique_featrue_val = df[featrue].unique() #df.unique：获取col特征下面所有可能的取值（不含重复值）
    res_dict = {ele:pd.DataFrame for ele in unique_featrue_val} #根据col不同取值作为键，值是空的数据框，构建数据字典
    
    #根据col特征的不同取值划分数据子集，分别放入对应的datafram中
    for key in res_dict.keys():
        res_dict[key] = df[:][df[featrue]==key]
    return res_dict

In [4]:
#对上一步的解释
import pandas as pd


df_1 = pd.DataFrame({'语文':[98,99,97,99,97,97],'数学':[99,89,88,99,88,99],'评价':['A','A','A','A','A','A']})
unique_col_val = df_1['语文'].unique()

print(unique_col_val)
#array([98, 99, 97], dtype=int64)

res_dict = {ele:pd.DataFrame for ele in unique_col_val}
print(res_dict)
'''
{98: pandas.core.frame.DataFrame,
 99: pandas.core.frame.DataFrame,
 97: pandas.core.frame.DataFrame}
'''


res_dict = df_split(df_1, '语文') #按照 语文 这一特征的取值划分df_1
print(res_dict)
'''
{98:    语文  数学
 0  98  99,
 99:    语文  数学
 1  99  89
 3  99  99,
 97:    语文  数学
 2  97  88
 4  97  88}
'''

[98 99 97]
{98: <class 'pandas.core.frame.DataFrame'>, 99: <class 'pandas.core.frame.DataFrame'>, 97: <class 'pandas.core.frame.DataFrame'>}
{98:    语文  数学 评价
0    98    99    A, 99:    语文  数学 评价
1    99    89    A
3    99    99    A, 97:    语文  数学 评价
2    97    88    A
4    97    88    A
5    97    99    A}


'\n{98:    语文  数学\n 0  98  99,\n 99:    语文  数学\n 1  99  89\n 3  99  99,\n 97:    语文  数学\n 2  97  88\n 4  97  88}\n'

In [5]:
#选择最优特征
def choose_best_feature(df, label):
    '''
    输入：
    df: 待划分的训练数据集
    label: 训练标签
    输出:
    max_vlaue:最大信息增益值
    best_fteture:最优特征
    max_splited:根据最优特征划分后的数据字典
    '''
    best_value, best_feature = -999, None
    best_splited = None
    
    #计算训练标签的信息熵
    entropy_D = entropy(df[label].tolist()) #计算当前数据集的信息熵, df[label].tolist(): ['no','no','yes','yes',...]
    
    #除标签以外的所有特征
    featrues = [featrue for featrue in df.columns if featrue not in label]
    
    for featrue in featrues:
        #根据col的取值划分数据集
        splited_set = df_split(df, featrue) #splited_set是按照col可能取值作为键的字典
        #初始化经验条件信息熵
        entropy_DA = 0
        for subset_featrue, subset in splited_set.items():
            entropy_Di = entropy(subset[label].tolist()) #计算当前子集的信息熵
            
            #用前一步计算的entropy_Di信息熵 * 子集的比例（也就是col当前取值占样本总数的比值）
            entropy_DA += len(subset)/len(df) * entropy_Di 
            
        #计算featrue当前取值下的信息增益
        info_gain = entropy_D - entropy_DA
        
        #找到最大信息增益的那一组：特征、信息增益值、数据集作为下一次切分的输入
        if info_gain > best_value:
            best_value, best_feature = info_gain, featrue
            best_splited = splited_set 
    return best_value, best_feature, best_splited

In [6]:
#featrues = [featrue for featrue in df.columns if featrue not in ['play']]
best_value, best_feature, best_splited = choose_best_feature(df_1, '评价')
best_value, best_feature, best_splited

(-0.0,
 '语文',
 {98:    语文  数学 评价
  0    98    99    A,
  99:    语文  数学 评价
  1    99    89    A
  3    99    99    A,
  97:    语文  数学 评价
  2    97    88    A
  4    97    88    A
  5    97    99    A})

In [7]:
best_splited[97]

,语文,数学,评价
2,97,88,A
4,97,88,A
5,97,99,A


In [8]:
columns_1 = ['数学','评价']
best_value_1, best_feature_1, best_splited_1 = choose_best_feature(best_splited[97][columns_1], '评价')
best_value_1, best_feature_1, best_splited_1

(-0.0,
 '数学',
 {88:    数学 评价
  2    88    A
  4    88    A,
  99:    数学 评价
  5    99    A})

In [9]:
columns_2 = ['评价']
best_value_2, best_feature_2, best_splited_2 = choose_best_feature(best_splited_1[88][columns_2], '评价')
best_value_2, best_feature_2, best_splited_2

(-999, None, None)

In [10]:
df = pd.read_csv(r'D:\\jupyter-notebook\\《机器学习：公式推导与代码实现》章节代码\\charpter7_decision_tree/example_data.csv', dtype={'windy': 'str'})
df[:5]

,humility,outlook,play,temp,windy
0,high,sunny,no,hot,false
1,high,sunny,no,hot,true
2,high,overcast,yes,hot,false
3,high,rainy,yes,mild,false
4,normal,rainy,yes,cool,false


In [11]:
best_value, best_feature, max_splited = choose_best_feature(df, 'play')
best_feature

'outlook'

In [12]:
class ID3Tree:
    class Node:
        def __init__(self, name):
            self.name = name
            self.connections = {}

        def connect(self, label, node):
            self.connections[label] = node
            
    def __init__(self, data, label): #data是一个df   label: play
        self.columns = data.columns
        self.data = data
        self.label = label
        self.root = self.Node("Root") #预设一个根节点
        
    def print_tree(self, node, tabs):
        print(tabs + node.name)
        for connection, child_node in node.connections.items():
            print(tabs + "\t" + "(" + str(connection) + ")")
            self.print_tree(child_node, tabs + "\t\t")

    def construct_tree(self):
        self.construct(self.root, "", self.data, self.columns)
        
        
        #parent_node的初始值：root, input_data初始值：data,也就是df
    def construct(self, parent_node, parent_connection_feature, input_data, columns):
        
        best_value, best_feature, best_splited = choose_best_feature(input_data[columns], self.label)
        print('此时max_value是{0}, best_col是{1}'.format(best_value, best_feature))
        print('==========================')
        print('max_splited是',best_splited)
        
        #这里划重点：书中的方式是穷尽所有特征，当所有特征被使用后，new_columns里面就是[]，所以执行到下一步的时候就会执行此句
        if not best_feature:
            node = self.Node(input_data[self.label].iloc[0])
            parent_node.connect(parent_connection_feature, node)
            print('没有合适的best_col, 此时的max_value, best_col是:',best_value, best_feature)
            print('==========================')
            print('max_splited是',best_splited)
            return # 等于return None 相当于结束调用

        node = self.Node(best_feature)
        parent_node.connect(parent_connection_feature, node)
        
        new_columns = [feature for feature in columns if feature != best_feature]
        
        for splited_value, splited_data in max_splited.items():
            print('当前除去best_col以外，还剩下的特征new_columns是:',new_columns)
            self.construct(node, splited_value, splited_data, new_columns)

In [13]:
tree1 = ID3Tree(df, 'play')
tree1.construct_tree()

此时max_value是0.2467498197744391, best_col是outlook
max_splited是 {'sunny':    humility outlook play  temp  windy
0      high   sunny   no   hot  false
1      high   sunny   no   hot   true
7      high   sunny   no  mild  false
8    normal   sunny  yes  cool  false
10   normal   sunny  yes  mild   true, 'overcast':    humility   outlook play  temp  windy
2      high  overcast  yes   hot  false
6    normal  overcast  yes  cool   true
11     high  overcast  yes  mild   true
12   normal  overcast  yes   hot  false, 'rainy':    humility outlook play  temp  windy
3      high   rainy  yes  mild  false
4    normal   rainy  yes  cool  false
5    normal   rainy   no  cool   true
9    normal   rainy  yes  mild  false
13     high   rainy   no  mild   true}
当前除去best_col以外，还剩下的特征new_columns是: ['humility', 'play', 'temp', 'windy']
此时max_value是0.9709505944546686, best_col是humility
max_splited是 {'high':   humility play  temp  windy
0     high   no   hot  false
1     high   no   hot   true
7     high   no 

In [14]:
tree1.print_tree(tree1.root, "  ")

  Root
  	()
  		outlook
  			(sunny)
  				humility
  					(sunny)
  						temp
  							(sunny)
  								windy
  									(sunny)
  										no
  									(overcast)
  										yes
  									(rainy)
  										yes
  							(overcast)
  								windy
  									(sunny)
  										no
  									(overcast)
  										yes
  									(rainy)
  										yes
  							(rainy)
  								windy
  									(sunny)
  										no
  									(overcast)
  										yes
  									(rainy)
  										yes
  					(overcast)
  						temp
  							(sunny)
  								windy
  									(sunny)
  										no
  									(overcast)
  										yes
  									(rainy)
  										yes
  							(overcast)
  								windy
  									(sunny)
  										no
  									(overcast)
  										yes
  									(rainy)
  										yes
  							(rainy)
  								windy
  									(sunny)
  										no
  									(overcast)
  										yes
  									(rainy)
  										yes
  					(rainy)
  						windy
  							(sun